### RC-GAN for ICU Data

Adapted from https://github.com/ratschlab/RGAN by Hyland, Esteban and Rätsch

See paper at https://arxiv.org/abs/1706.02633


In [28]:
import numpy as np
import tensorflow as tf
import data_utils
import model

tf.logging.set_verbosity(tf.logging.ERROR)

batch_size = 28
seq_length = 14
latent_dim = 5
num_signals = 14
cond_dim = 0
batch_mean = False
hidden_units_d = 100
hidden_units_g = 100
num_generated_features = 14
learn_scale = True
kappa = 1
wrong_labels = False

In [2]:

data_settings = {
    'scale': 0.1,
    'full_mnist': False}
samples, pdf, labels = data_utils.get_data('sine', data_settings)


Generated/loaded 14000 samples from data-type sine


In [7]:
norm = False
if labels is None:
    train, vali, test = data_utils.split(samples, [0.6, 0.2, 0.2], normalise=norm)
    train_labels, vali_labels, test_labels = None, None, None
else:
    train, vali, test, labels_list = data_utils.split(samples, [0.6, 0.2, 0.2], normalise=norm, labels=labels)
    train_labels, vali_labels, test_labels = labels_list

In [11]:
data_path = 'sine.data.npy'
np.save(data_path, {'samples': samples, 'pdf': pdf, 'labels': labels})
print('Saved training data to', data_path)

Saved training data to sine.data.npy


### Model Building

In [22]:
Z, X, CG, CD, CS = model.create_placeholders(batch_size, seq_length, latent_dim, 
                                    num_signals, cond_dim)

discriminator_settings = {
    'hidden_units_d': hidden_units_d ,
    'seq_length' : seq_length,
    'cond_dim':cond_dim,
    'batch_size':batch_size, 
    'batch_mean':batch_mean
}
generator_settings = {
    'hidden_units_g': hidden_units_g,
    'seq_length': seq_length,
    'batch_size':batch_size, 
    'num_generated_features':num_generated_features, 
    'cond_dim':cond_dim,
    'learn_scale': learn_scale
}


In [29]:
CGAN = (cond_dim > 0)
if CGAN: assert not predict_labels
D_loss, G_loss = model.GAN_loss(Z, X, generator_settings, discriminator_settings, kappa, CGAN, CG, CD, CS, wrong_labels=wrong_labels)
D_solver, G_solver = model.GAN_solvers(D_loss, G_loss, learning_rate)
G_sample = model.generator(Z, **generator_settings, reuse=True, c=CG)


AssertionError: 